# Findings

Here we'll hopefully answer all the questions raised before and during the exploration phase, and present them in some funky visuals.

> Plan
> - Team toss success and decsions
> - Team win %, bat 1st, toss
> - Team score vs RPO, with pearsons coefficient
> - Matches played in April heatmap
> - Australia losing games with higher score average
> - Hypothesese:
        - winning the toss is worse than losing the toss
        - batting first is worse than batting second
        - win % after winning / losing previous match